# K (Chebeshev-order) changes in 1 GCN conv layer

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')

from lib import models, graph, coarsening, utils
import tensorflow as tf
import numpy as np
import time
import scipy.sparse
from scipy.io import *
%matplotlib inline


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Graphs.
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
# TODO: change cgcnn for combinatorial Laplacians.
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

# Directories.
flags.DEFINE_string('dir_data', os.path.join('..', 'data', 'mnist'), 'Directory to store data.')

In [ ]:
# Number of Nearest Neighbors
k = 110

base_storage_dir = '/data4/abhijeet/Datasets/PASCAL_VOC/GCN/preprocessing/experiment3/'
temp_filename = base_storage_dir + 'scikit_fc7_k_' + str(k) + '.mat'

if os.path.exists(temp_filename):
    print('File already exists\n No Preprocessing Required')
else:
    print('Do preprocessing --> TODO --> call the fucntion with a specific parameter')
    
data = scipy.io.loadmat( temp_filename)

X_train          = data['X_train']
X_test           = data['X_test']
train_labels     = data['Y_train']
test_labels      = data['Y_test']
Adjacency_test   = data['Adjacency_test']
Adjacency_train  = data['Adjacency_train']

#delete unused variables
del data

File already exists
 No Preprocessing Required


In [ ]:
#define laplacians, coarsen adjacency, permutate data points.
train_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]
test_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]

train_perm ,test_perm = [] ,[]

for i in range(X_train.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_train[0][i], levels=FLAGS.coarsening_levels, self_connections=False)
    train_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        train_laplacians[j].append(temp_L[j])     

for i in range(X_test.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_test[0][i], levels=FLAGS.coarsening_levels, self_connections=False)
    test_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        test_laplacians[j].append(temp_L[j])     

del Adjacency_test, Adjacency_train

0
1000


In [ ]:
print(len(train_perm[0]),type(train_perm))
train_data = np.zeros((X_train.shape[0], len(train_perm[0]), X_train.shape[2]))
test_data = np.zeros((X_test.shape[0], len(test_perm[0]), X_test.shape[2]))
for i in range(train_data.shape[0]):
    if not(i%1000):
        print(i)
    train_data[i,:,:] = coarsening.perm_data_point(X_train[i,:,:], train_perm[i])

for i in range(test_data.shape[0]):
    if not(i%1000):
        print(i)
    test_data[i,:,:] = coarsening.perm_data_point(X_test[i,:,:], test_perm[i])

val_data = test_data.copy()
val_laplacians = test_laplacians.copy()
val_labels = test_labels.copy()

del X_train, X_test, train_perm, test_perm

In [ ]:
print(len(test_laplacians))
#print(test_laplacians[0])
L = []
for i in range(FLAGS.coarsening_levels):
    L.append(train_laplacians[i][0])
L = np.array(L)
for i in range(4):
    print(L[i].shape)

In [ ]:
common = {}
common['dir_name']       = 'pascal_voc_experiment_2/'
common['num_epochs']     = 200
common['batch_size']     = 100
common['decay_steps']    = (train_data.shape[0] + val_data.shape[0]) / common['batch_size']
common['eval_frequency'] = 10 * common['num_epochs']
common['brelu']          = 'b1relu'
common['pool']           = 'mpool1'
num_labels_per_image     = 2
C = train_labels.shape[1]  # number of classes
model_perf = utils.model_perf()

In [ ]:
# Common hyper-parameters for networks with one convolutional layer.
common['regularization'] = 0
common['dropout']        = 0.5
common['learning_rate']  = 0.2
common['decay_rate']     = 0.95
common['momentum']       = 0.9
common['F']              = [10]
common['F_0']            = 20
common['p']              = [4]
common['M']              = [C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians

In [ ]:
common['K']              = [10]
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

In [ ]:
common['K']              = [30]
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

In [ ]:
common['K']              = [60]
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

In [ ]:
common['K']              = [100]
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

In [ ]:
common['K']              = [200]
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)